<a href="https://colab.research.google.com/github/muffindonor/Cloud-Computing-Exercise-repo/blob/master/Ex4_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import requests
import plotly.graph_objects as go
from ipywidgets import widgets, Output, VBox, Tab
from IPython.display import display, clear_output

# --- שליפת נתונים מהמאגר הממשלתי ---
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3&limit=10000'
response = requests.get(url)
data = response.json()
data_df = pd.DataFrame(data['result']['records'])

# --- טאב 1: סטטיסטיקות כלליות ---
def create_statistics_tab():
    output = Output()
    with output:
        clear_output(wait=True)
        display(widgets.HTML("<h3>סטטיסטיקות כלליות</h3>"))

        # המרה בטוחה של עמודת השנה למספרים
        data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')

        total_records = len(data_df)
        unique_makers = data_df['tozeret_nm'].nunique()
        unique_models = data_df['kinuy_mishari'].nunique()
        years = data_df['shnat_yitzur'].dropna().unique()

        stats_html = f"""
        <p><b>סך הרשומות:</b> {total_records}</p>
        <p><b>יצרנים ייחודיים:</b> {unique_makers}</p>
        <p><b>דגמים ייחודיים:</b> {unique_models}</p>
        <p><b>שנות ייצור:</b> {', '.join(map(str, sorted(map(int, years))))}</p>
        """
        display(widgets.HTML(stats_html))
    return output

# --- טאב 2: טבלה של המידע ---
def create_table_tab():
    output = Output()
    with output:
        clear_output(wait=True)
        display(widgets.HTML("<h3>טבלת נתונים מלאה</h3>"))
        display(data_df.head(20))  # מציג את 20 השורות הראשונות
    return output

# --- טאב 3: גרף לפי שנות ייצור ---
def create_graph_tab():
    output = Output()
    with output:
        clear_output(wait=True)
        display(widgets.HTML("<h3>מספר כלי רכב לפי שנת ייצור</h3>"))

        # המרה בטוחה של עמודת השנה למספרים
        data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')

        # סינון נתונים תקינים
        years_count = data_df.groupby('shnat_yitzur').size().reset_index(name='count')
        years_count = years_count.dropna()  # הסרת שורות עם ערכים חסרים

        # יצירת גרף עמודות עם Plotly
        fig = go.Figure(
            data=[go.Bar(
                x=years_count['shnat_yitzur'],
                y=years_count['count'],
                marker_color='rgb(58,200,225)'
            )]
        )

        fig.update_layout(
            title="מספר כלי רכב לפי שנת ייצור",
            xaxis_title="שנה",
            yaxis_title="מספר כלי רכב",
            bargap=0.2
        )

        # הצגת הגרף
        fig.show()

    return output

# --- טאב 4: סינון יצרן, דגם ורמת גימור ---
def create_filter_tab():
    output = Output()
    tozeret_dropdown = widgets.Dropdown(
        options=['All'] + list(data_df['tozeret_nm'].dropna().unique()),
        description='יצרן:'
    )
    kinuy_dropdown = widgets.Dropdown(description='דגם:')
    trim_output = Output()

    def update_models(change):
        chosen_maker = tozeret_dropdown.value
        if chosen_maker == 'All':
            kinuy_dropdown.options = ['All']
        else:
            models = data_df[data_df['tozeret_nm'] == chosen_maker]['kinuy_mishari'].dropna().unique()
            kinuy_dropdown.options = ['All'] + list(models)

    def update_trim_levels(change):
        chosen_maker = tozeret_dropdown.value
        chosen_model = kinuy_dropdown.value
        with trim_output:
            clear_output(wait=True)
            if chosen_maker == 'All' or chosen_model == 'All':
                display(widgets.HTML("<p>אנא בחר יצרן ודגם להצגת פרטים.</p>"))
            else:
                filtered = data_df[(data_df['tozeret_nm'] == chosen_maker) & (data_df['kinuy_mishari'] == chosen_model)]
                trim_levels = filtered['ramat_gimur'].dropna().unique()
                total_records = len(filtered)
                display(widgets.HTML(f"<p><b>סך רשומות:</b> {total_records}</p>"))
                display(widgets.HTML(f"<p><b>רמות גימור ייחודיות:</b> {', '.join(trim_levels)}</p>"))

    tozeret_dropdown.observe(update_models, names='value')
    kinuy_dropdown.observe(update_trim_levels, names='value')

    with output:
        display(VBox([tozeret_dropdown, kinuy_dropdown, trim_output]))
    return output

# --- יצירת הטאבים ---
tab = Tab()
tab_children = [
    create_statistics_tab(),
    create_table_tab(),
    create_graph_tab(),
    create_filter_tab()
]
tab.children = tab_children
tab.set_title(0, "סטטיסטיקות")
tab.set_title(1, "טבלת נתונים")
tab.set_title(2, "גרף")
tab.set_title(3, "סינון")

# --- הצגת הטאבים ---
display(tab)

In [8]:
import pandas as pd
import plotly.express as px

data = {
    "Country": ["Germany", "France", "Italy", "Spain", "United Kingdom"],
    "GDP": [3.8,2.7,1.9,1.4,2.1],
    "Population": [83,67,60,47,66],
    "Vehicles Fleet": [50,35,40,30,45]
}

df = pd.DataFrame(data)

fig = px.scatter_matrix(df,
                        dimensions = ["GDP", "Population", "Vehicles Fleet"],
                        color = "Country",
                        title = "Scatter Matrix: Country GDP, Population, and Vehicles Fleet in Europe")

fig.show()

In [10]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = ["#%02x%02x%02x" % (r, g, 150) for r, g in zip(np.floor(50 + 2 * x).astype(int), np.floor(30 + 2 * y).astype(int))]

p = figure()
p.circle(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)
show(p)


In [11]:
#Plotly creates interactive visualizations that work wonderfully in Colab's notebook environment -you can hover over data points, zoom in/out, and pan around the plots.
#One of Plotly's great features in Colab is that the plots are rendered natively without any additional display configuration needed
from plotly.offline import iplot
import plotly.graph_objs as go
data = [
go.Contour(
z=[[10, 10.625, 12.5, 15.625, 20],
[5.625, 6.25, 8.125, 11.25, 15.625],
[2.5, 3.125, 5., 8.125, 12.5],
[0.625, 1.25, 3.125, 6.25, 10.625],
[0, 0.625, 2.5, 5.625, 10]]
)
]
iplot(data)

In [12]:
import plotly.graph_objects as go
import pandas as pd
df1 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/718417069ead87650b90472464c7565dc8c2cb1c/sunburst-coffee-flavors-complete.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/718417069ead87650b90472464c7565dc8c2cb1c/coffee-flavors.csv')
fig = go.Figure()
fig.add_trace(go.Sunburst(
ids=df1.ids,
labels=df1.labels,
parents=df1.parents,
domain=dict(column=0)
))
fig.add_trace(go.Sunburst(
ids=df2.ids,
labels=df2.labels,
parents=df2.parents,
domain=dict(column=1),
maxdepth=2
))
fig.update_layout(
grid= dict(columns=2, rows=1),
margin = dict(t=0, l=0, r=0, b=0)
)
fig.show()

In [13]:
import plotly.graph_objects as go
import datetime
import numpy as np
np.random.seed(1)
programmers =['Alex','Nicole','Sara','Etienne','Chelsea','Jody','Marianne']
base =datetime.datetime.today()
dates =base -np.arange(180) *datetime.timedelta(days=1)
z =np.random.poisson(size=(len(programmers), len(dates)))
fig =go.Figure(data=go.Heatmap(
z=z,
x=dates,
y=programmers,
colorscale='Viridis'))
fig.update_layout(
title=dict(text='GitHub commits per day'),
xaxis_nticks=36)
fig.show()

In [16]:
# Plotly version
import plotly.express as px
df = pd.DataFrame({'x': range(10), 'y': [i**2 for i in range(10)]})
fig = px.scatter(df, x='x', y='y', title='Plotly Scatter')
fig.show()
# Bokeh version
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure(title='Bokeh Scatter')
p.scatter(df['x'], df['y'])
show(p)

In [22]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns #seaborn for heatmap

# Create a simple DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
}
df = pd.DataFrame(data)

# Tab 1: Data Overview
tab1_content = widgets.Output()
with tab1_content:
    print("Data Overview:")
    display(df.describe())  # Summary statistics

# Tab 2: Raw Data
tab2_content = widgets.Output()
with tab2_content:
    print("Raw Data:")
    display(df)  # Full DataFrame

# Tab 3: Charts
tab3_content = widgets.Output()
with tab3_content:
    print("Charts:")
    plt.clf()  # Clear the current figure
    plt.figure(figsize=(10, 8))  # Increased figure size to accommodate labels
    side_length = 10
    data = 5 + np.random.randn(side_length, side_length)
    data += np.arange(side_length)
    data += np.reshape(np.arange(side_length), (side_length, 1))

    # Create heatmap with labels
    heatmap = sns.heatmap(data,
                          cmap='viridis',
                          annot=True,
                          fmt='.1f',
                          cbar_kws={'label': 'Value Intensity'})
    plt.title('Enhanced Heatmap with Labels')
    plt.xlabel('X Axis Position')
    plt.ylabel('Y Axis Position')
    plt.show()  # Add this to display the plot
    plt.close()  # Close the figure to free memory

# Create tabs for navigation
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content])
tabs.set_title(0, 'Data Overview')
tabs.set_title(1, 'Raw Data')
tabs.set_title(2, 'Charts')

# Display the tabs
display(tabs)
